# P1. Clasificador de ropa

In [2]:
import tensorflow as tf
tf.__version__ >= '2.0.0'

True

In [12]:
from keras.datasets import fashion_mnist
(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()

4422102/4422102 [==============================] - 0s 0us/step
